In [1]:
import pandas as pd
import numpy as np
import math
import json
import os
import scipy.io, random
from pathlib import Path
import itertools

abs_path = os.getcwd()

data_path = str(Path().resolve().parent) + "/PPI_sequence/"

In [2]:
data_path

'/mnt/a8fe2bcf-9383-4903-9155-05f77b84ac2b/DensePPI/Datasets/Independent Datasets - PanEtAl/PPI_sequence/'

In [3]:
PAmat = scipy.io.loadmat(data_path + 'Mix_Hpylo_data.mat')
PBmat = scipy.io.loadmat(data_path + 'Mix_Hpylo_data.mat')

In [4]:
P1 = [PAmat['protein_A'][i, 0][0] for i in range(len(PAmat['protein_A']))]
P2 = [PBmat['protein_B'][i, 0][0] for i in range(len(PBmat['protein_B']))]

In [5]:
unique = list(dict.fromkeys(list(itertools.chain(P1, P2))))

In [6]:
protdict = {}
for i in range(len(unique)):
    protdict[unique[i]] = "P" + str(i)

In [7]:
PA = [protdict[prot] for prot in P1]
PB = [protdict[prot] for prot in P2]

In [8]:
#rand_nums = random.sample(list(range(len(NAmat['N_protein_a']))), len(PAmat['P_protein_a']))
#N1 = [NAmat['N_protein_a'][i, 0][0] for i in rand_nums]
#N2 = [NBmat['N_protein_b'][i, 0][0] for i in rand_nums]

In [9]:
pdict = [{'pA': pA, 'pB': pB} for pA, pB in zip(PA, PB)]

In [10]:
protdict_map = {v: k for k, v in protdict.items()}

In [11]:
with open("positive.json", 'w') as outfile:
    json.dump(pdict, outfile)   

In [12]:
df_pos = pd.read_pickle(str(Path().resolve().parent.parent) + "/Benchmark/PanEtAl/positive.pkl")
df_neg = pd.read_pickle(str(Path().resolve().parent.parent) + "/Benchmark/PanEtAl/negative.pkl")

total_proteins_bench = set(df_neg['Protein_1']) | set(df_pos['Protein_1']) | set(df_neg['Protein_2']
                                                                                ) | set(df_pos['Protein_2'])

total_proteins_bench = list(total_proteins_bench)
len(total_proteins_bench)

new_df = df_neg[['Protein_1', 'Protein_seq1']]
new_df.columns = ['ID', 'Sequence']
temp = df_neg[['Protein_2', 'Protein_seq2']]
temp.columns = ['ID', 'Sequence']
new_df = new_df.append(temp, ignore_index = True)
temp = df_pos[['Protein_1', 'Protein_seq1']]
temp.columns = ['ID', 'Sequence']
new_df = new_df.append(temp, ignore_index = True)
temp = df_pos[['Protein_2', 'Protein_seq2']]
temp.columns = ['ID', 'Sequence']
new_df = new_df.append(temp, ignore_index = True)

benchdict = new_df.set_index('ID')['Sequence'].to_dict()

In [13]:
res = {**benchdict, **protdict_map}

ofile = open("bench_hpylori.fasta", "w")

for protein in res.keys():
    ofile.write(">" + protein + "\n" + res[protein] + "\n")

ofile.close()

In [14]:
with open("seq.json", 'w') as outfile:
    json.dump(res, outfile)

# For 40%
http://weizhong-lab.ucsd.edu/cdhit_suite/cgi-bin/result.cgi?JOBID=1654178430